# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
# load data from database
engine = create_engine('sqlite:///DataBase.db')
df = pd.read_sql('SELECT * FROM df', engine)

In [4]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

#### cleaning non-binary columns:

In [6]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [7]:
bad_samples = []
for i in df.index:
    if df.loc[i,'related'] == 2:
        bad_samples.append(i)

In [8]:
len(bad_samples)

188

In [9]:
good_samples = [i for i in df.index if i not in bad_samples ]

In [10]:
# checking:
len(good_samples)+len(bad_samples), df.shape[0]

(26216, 26216)

In [11]:
df = df.iloc[good_samples,]
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [12]:
# final check:
df.related.value_counts()

1    19906
0     6122
Name: related, dtype: int64

#### splitting  feature vs target:

In [13]:
features = ['message']

In [16]:
X = df[features]
X = X['message']
X.head(3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

In [17]:
non_target_cols = ['id','message','original','genre'] 
non_target_cols

['id', 'message', 'original', 'genre']

In [18]:
target_cols = df.columns[~df.columns.isin(non_target_cols)]
target_cols

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [19]:
Y = df[target_cols]
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [20]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [21]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [22]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [24]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize) ),
    ('tfidf', TfidfTransformer() ),
    ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [27]:
(X_train.shape , Y_train.shape)

((19521,), (19521, 36))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [28]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [29]:
from sklearn.multioutput import MultiOutputClassifier

In [30]:
clf = pipeline.fit(X_train, Y_train)

In [31]:
Y_pred = pipeline.predict(X_test)

In [32]:
(Y_test.shape, Y_pred.shape)

((6507, 36), (6507, 36))

In [33]:
[type(Y_test), type(Y_pred)]

[pandas.core.frame.DataFrame, numpy.ndarray]

In [34]:
Y_pred = pd.DataFrame(Y_pred,columns=Y_test.columns)
Y_pred.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [35]:
import numpy as np

In [36]:
from sklearn.utils.multiclass import type_of_target

In [37]:
[type_of_target(Y_test), type_of_target(Y_pred)]

['multilabel-indicator', 'multilabel-indicator']

In [38]:
import inspect
inspect.signature(classification_report)

<Signature (y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2)>

In [39]:
print(classification_report(Y_test,Y_pred,target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4981
               request       0.77      0.41      0.53      1114
                 offer       0.00      0.00      0.00        31
           aid_related       0.73      0.61      0.66      2655
          medical_help       0.50      0.08      0.14       522
      medical_products       0.66      0.08      0.14       342
     search_and_rescue       0.40      0.04      0.07       164
              security       0.50      0.01      0.01       132
              military       0.60      0.12      0.20       208
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.44      0.58       440
                  food       0.83      0.55      0.66       693
               shelter       0.81      0.41      0.54       606
              clothing       1.00      0.05      0.09       107
                 money       0.75      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [40]:
print(f1_score((Y_test), (Y_pred),average='macro'))

0.246561690716


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# parameters in CountVectorizer
inspect.signature(CountVectorizer)

In [ ]:
# parameters in TfidfTransformer
inspect.signature(TfidfTransformer)

In [ ]:
# parameters in RandomForestClassifier
inspect.signature(RandomForestClassifier)

In [41]:
list(pipeline.get_params().keys())

['memory',
 'steps',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__bootstrap',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimat

In [42]:
parameters = {
    'vect__tokenizer': [tokenize,None], # for count vectorizer
    'tfidf__norm': ['l1','l2'], # for tfidf
    'clf__estimator__criterion': ['gini','entropy'] # for Random Forest
}

# Note: using 'clf__criterion' as syntax generates an error:
# ValueError: Invalid parameter criterion for estimator MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
# By doing cv.get_params().keys() one can see the above one is the right syntax

In [43]:
from sklearn.model_selection import GridSearchCV

In [ ]:
inspect.signature(GridSearchCV)

In [44]:
cv = GridSearchCV(pipeline, param_grid=parameters)

In [45]:
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__tokenizer': [<function tokenize at 0x7f583a126a60>, None], 'tfidf__norm': ['l1', 'l2'], 'clf__estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [46]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [47]:
pd.DataFrame(cv.cv_results_)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__criterion,param_tfidf__norm,param_vect__tokenizer,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,39.574330,0.252547,4.464049,0.098948,gini,l1,<function tokenize at 0x7f583a126a60>,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.233134,0.233134,0.244506,0.236924,0.005361,4,0.803827,0.803750,0.807208,0.804928,0.001612
1,32.335796,0.535433,1.681924,0.013685,gini,l1,None,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.213923,0.221454,0.222222,0.219200,0.003744,7,0.810973,0.807054,0.811050,0.809692,0.001866
2,38.549490,0.469520,4.416410,0.035773,gini,l2,<function tokenize at 0x7f583a126a60>,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.237437,0.232365,0.246350,0.238717,0.005781,2,0.811280,0.810589,0.804749,0.808872,0.002930
3,32.201870,0.238465,1.698327,0.020997,gini,l2,None,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.218227,0.217458,0.230521,0.222069,0.005985,6,0.808591,0.808053,0.809743,0.808796,0.000705
4,40.529523,0.443410,4.085497,0.054451,entropy,l1,<function tokenize at 0x7f583a126a60>,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.239281,0.232519,0.245274,0.239025,0.005211,1,0.807361,0.806670,0.807669,0.807233,0.000418
5,33.222514,0.280653,1.297310,0.011502,entropy,l1,None,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.224681,0.214999,0.228062,0.222581,0.005536,5,0.802905,0.799831,0.802136,0.801624,0.001306
6,40.911137,0.571830,4.072481,0.045474,entropy,l2,<function tokenize at 0x7f583a126a60>,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.239742,0.235900,0.240510,0.238717,0.002017,2,0.812202,0.808207,0.805363,0.808591,0.002805
7,33.120191,0.218528,1.299065,0.002380,entropy,l2,None,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.212079,0.214538,0.221300,0.215973,0.003899,8,0.802213,0.808744,0.801982,0.804313,0.003135


In [48]:
cv.best_score_

0.23902464013114083

In [49]:
cv.best_params_

{'clf__estimator__criterion': 'entropy',
 'tfidf__norm': 'l1',
 'vect__tokenizer': <function __main__.tokenize(text)>}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [53]:
Y_pred_cv = cv.best_estimator_.predict(X_test)

In [51]:
print(classification_report(Y_test,Y_pred_cv,target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.89      4981
               request       0.82      0.42      0.55      1114
                 offer       0.00      0.00      0.00        31
           aid_related       0.74      0.61      0.67      2655
          medical_help       0.58      0.08      0.14       522
      medical_products       0.64      0.03      0.05       342
     search_and_rescue       1.00      0.06      0.11       164
              security       0.50      0.01      0.01       132
              military       0.54      0.03      0.06       208
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.29      0.44       440
                  food       0.78      0.52      0.62       693
               shelter       0.79      0.26      0.39       606
              clothing       0.86      0.06      0.11       107
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [52]:
print(f1_score(Y_test,Y_pred,average='macro'))

0.246561690716


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [54]:
from sklearn.ensemble import GradientBoostingClassifier

In [55]:
from sklearn.neighbors import KNeighborsClassifier

##### verb extractor method from class

In [56]:
from sklearn.base import BaseEstimator, TransformerMixin
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [57]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [58]:
## from Mentor:

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [59]:
from sklearn.pipeline import FeatureUnion

In [60]:
pipeline_improved = Pipeline([
    ('features', FeatureUnion([
        ('nlp_pipeline',  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize) ),
            ('tfidf', TfidfTransformer() )
                                  ]) ),
        ('txt_extract', StartingVerbExtractor() )
                            ]) ),      
        
    ('knn', MultiOutputClassifier(KNeighborsClassifier() ) )
                            ])

In [61]:
list(pipeline_improved.get_params().keys())

['memory',
 'steps',
 'features',
 'knn',
 'features__n_jobs',
 'features__transformer_list',
 'features__transformer_weights',
 'features__nlp_pipeline',
 'features__txt_extract',
 'features__nlp_pipeline__memory',
 'features__nlp_pipeline__steps',
 'features__nlp_pipeline__vect',
 'features__nlp_pipeline__tfidf',
 'features__nlp_pipeline__vect__analyzer',
 'features__nlp_pipeline__vect__binary',
 'features__nlp_pipeline__vect__decode_error',
 'features__nlp_pipeline__vect__dtype',
 'features__nlp_pipeline__vect__encoding',
 'features__nlp_pipeline__vect__input',
 'features__nlp_pipeline__vect__lowercase',
 'features__nlp_pipeline__vect__max_df',
 'features__nlp_pipeline__vect__max_features',
 'features__nlp_pipeline__vect__min_df',
 'features__nlp_pipeline__vect__ngram_range',
 'features__nlp_pipeline__vect__preprocessor',
 'features__nlp_pipeline__vect__stop_words',
 'features__nlp_pipeline__vect__strip_accents',
 'features__nlp_pipeline__vect__token_pattern',
 'features__nlp_pipeli

In [62]:
knn = pipeline_improved.fit(X_train, Y_train)

In [63]:
knn.score

<function sklearn.pipeline.Pipeline.score(self, X, y=None, sample_weight=None)>

In [64]:
for att in dir(knn):
    if not att.startswith('_'):
        print(att)

classes_
decision_function
fit
fit_predict
fit_transform
get_params
inverse_transform
memory
named_steps
predict
predict_log_proba
predict_proba
score
set_params
steps
transform


In [65]:
Y_pred_improved = pipeline_improved.predict(X_test)

In [66]:
print(classification_report(Y_test,Y_pred_improved,target_names=Y_test.columns))


                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4981
               request       0.68      0.43      0.52      1114
                 offer       0.00      0.00      0.00        31
           aid_related       0.68      0.43      0.53      2655
          medical_help       0.63      0.10      0.17       522
      medical_products       0.61      0.11      0.19       342
     search_and_rescue       0.56      0.05      0.10       164
              security       0.00      0.00      0.00       132
              military       0.72      0.10      0.18       208
           child_alone       0.00      0.00      0.00         0
                 water       0.71      0.22      0.33       440
                  food       0.67      0.31      0.43       693
               shelter       0.64      0.15      0.24       606
              clothing       0.67      0.11      0.19       107
                 money       0.61      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [68]:
score = f1_score(Y_test,Y_pred_improved, average='macro')
score

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.19827520437555368

### Pipeline with two estimators:

### 9. Export your model as a pickle file

In [69]:
import pickle

In [70]:
pickle.dump(knn,open('knn.pkl','wb'))

In [71]:
pickle.dump(cv.best_estimator_,open('rfc_cv.pkl','wb'))

In [72]:
pickle.dump(clf,open('clf.pkl','wb'))

In [ ]:
import joblib

In [ ]:
joblib.dump( , "classifier.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Testing `train_classifier.py` :

In [1]:
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
import pickle
import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
database_filepath, model_filepath = sys.argv[1:]

In [25]:
model_filepath

'/root/.local/share/jupyter/runtime/kernel-ec344199-f04e-4ab8-b530-50f52ddffac0.json'

In [24]:
database_filepath

'-f'

#### load_data()

In [2]:
def load_data(database_filepath):
    """
    Loads data from SQL Database and transforms it for model training
    
    :param database_filepath: SQL database file (string)
    
    :returns x: Features (dataframe)
    :returns y: Targets (dataframe)
    :returns category_names: Target labels (list)
    """
    
    engine = create_engine('sqlite:///{}'.format(database_filepath))
    df = pd.read_sql('SELECT * FROM df', engine)
    
    bad_samples = []
    for i in df.index:
        if df.loc[i,'related'] == 2:
            bad_samples.append(i)
    good_samples = [i for i in df.index if i not in bad_samples ]   
    df = df.iloc[good_samples,]   

    features = ['message']
    X = df[features]
    X = X['message']
    
    non_target_cols = ['id','message','original','genre'] 
    target_cols = df.columns[~df.columns.isin(non_target_cols)]
    Y = df[target_cols]
    
    category_names = Y.columns
    return X,Y, category_names

In [3]:
X, Y, category_names = load_data('DataBase.db')

In [4]:
X.head(3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

In [5]:
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

#### buil_model()

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [9]:
X_test.shape, Y_test.shape

((5206,), (5206, 36))

In [8]:
def tokenize(text):
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens

In [10]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged) 

In [12]:
def build_model():
    """
    Builds a model using Random Forest Classifier. Data is transformed in pipeline using Tokenization, Count Vectorizer,
    Tfidf Transformer and
    
    :return cv: Trained model after performing grid search (GridSearchCV model)
    """
    
    pipeline = Pipeline([
    ('features', FeatureUnion([
        ('nlp_pipeline',  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize) ),
            ('tfidf', TfidfTransformer() )]) ),
        ('txt_extract', StartingVerbExtractor() )]) ),          
    ('clf', MultiOutputClassifier(RandomForestClassifier() ) )])
    
    parameters = {
    'features__nlp_pipeline__vect__tokenizer': [tokenize,None], # for count vectorizer
    'features__nlp_pipeline__tfidf__norm': ['l1','l2'], # for tfidf
    'clf__estimator__criterion': ['gini','entropy']} # for Random Forest
    
    #cv = GridSearchCV(estimator=pipeline, param_grid=parameters,scoring='f1_macro')
    
    cv = pipeline
    return cv

In [13]:
model = build_model()

In [14]:
model

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

#### evaluate_model():

In [15]:
X_train.shape, Y_train.shape

((20822,), (20822, 36))

In [17]:
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [20]:
def evaluate_model(model, X_test, Y_test, category_names):                     
    """
    Measures model's performance on test data and prints out results.
    :param model: trained model (GridSearchCV Object)
    :param X_test: Test features (dataframe)
    :param Y_test: Test targets (dataframe)
    :param category_names: Target labels (dataframe)
    
    :return model_report: Dataframe with Model Performance report
    """             
    Y_pred = model.predict(X_test) #best_estimator_
    Y_pred = pd.DataFrame(Y_pred,columns=Y_test.columns)
    #print("\nBest Parameters:", model.best_params_)
    #score = f1_score( Y_test,Y_pred, average='macro')
    
    return print(classification_report(Y_test,Y_pred,target_names=category_names))

In [21]:
evaluate_model(model, X_test, Y_test, category_names)

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      3974
               request       0.80      0.45      0.57       887
                 offer       0.00      0.00      0.00        21
           aid_related       0.75      0.60      0.67      2162
          medical_help       0.55      0.09      0.15       410
      medical_products       0.66      0.09      0.15       265
     search_and_rescue       0.33      0.01      0.01       137
              security       0.00      0.00      0.00        86
              military       0.71      0.09      0.16       187
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.27      0.41       345
                  food       0.86      0.54      0.66       578
               shelter       0.78      0.28      0.41       446
              clothing       0.60      0.08      0.14        78
                 money       0.86      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [30]:
model_filepath = 'classifier.pkl'

In [28]:
def save_model(model, model_filepath):
    """
    Function to save trained model as pickle file.
    :param model: Trained model (GridSearchCV Object)
    :param model_filepath: Filepath to store model (string)
    :return: None
    """
    pickle.dump(model, open(model_filepath, 'wb'))
    #pickle.dump(model,model_filepath)

In [31]:
save_model(model, model_filepath)

In [ ]:
# load data from database
engine = create_engine('sqlite:///DataBase.db')
df = pd.read_sql('SELECT * FROM df', engine)